# Overview of Functions

To access all, `%run A_FunctionDefinitions.ipynb ;` 

### [Pre-Requesite Functions](#Setting-Pre-requisite-Functions-/-Importing-Libraries)
`data=loadflow(gauge)` Returns dataframe directly loading the CAMELS provided streamflow data

`fdc=FDC(gauge)` Returns dataframe with FDC info - Flow, Count Exceeded, Percent Exceeded

### [FDC Plotting](#FDC-Plots)
`fig=FDC_breakpoints(gauge)` Finds optimal breakpoint for 2 line representation of FDC, plots it

`fig=MedianFDC(gauge)` Returns plot of each year's FDC, median annual FDC, and regular FDC

`fig=MedianScaledFDC(gauge)` Returns plot of each year's FDC scaled to annual means, median of that, and regular FDC

`fig=PeriodHydrographFDC(gauge)` Plots period of record FDC overlayed on Period Hydrograph

`fig=HydrographFDCOverlay(gauge)` Overlays FDC and annual min/med/max hydrograph

### [Other Streamflow Plotting](#Other-Streamflow-Plots)
`fig=RasterHydrograph(gauge)` Return image of raster hydrograph, with no data in black

`fig=StackedHydrograph(gauge)` Returns plot of each year's hydrograph year with min/med/max values for each day

`fig=TypicalYear(gauge)` Returns plot of typical year with min/med/max values for each day

`multiplot(gauge)` MapMaker, TypicalYear, RasterHydrograph, PeriodHydrographFDC

### [Mapping](#I'M-THE-MAP!)
`CONUSplot()` Creates plain map of the continental US

`fig=MapMaker(gauge)` Returns map of CONUS with red dot at gauge location

`mapdata(category,column,colorscale='divergent',gauge=[],subset=[],show=1)`

### [Emphasize Selected Years](#Plots-to-Emphasize-Selected-Years)
Note: `selected_years` should be bracketed list, even if only one year (ex: `=[1985]`)

`fig=MedianFDC_selectyears(gauge,selected_years)` Returns plot specific year FDCs over all FDCs, median FDC, and regular FDC 

`fig=MedianScaledFDC_selectyears(gauge,selected_years)` Returns plot specific year scaled FDCs over all scaled FDCs, median FDC, and regular FDC 

`fig=TypicalYear_selectyears(gauge,selected_years)` Returns hydrograph of chosen years over period of record min/med/max 

### [Return Attribute Data](#Returning-Attribute-Data)
`returnattributes(gauge)` Displays all CAMELS provided attributes, along with two sets of calculated streamflow figures

`data=returnattributeset(dataset)`  Returns full dataframe of attributes/signatures 

### [Comparison Plots](#Some-Comparison-Plots)
`fig=comparisonplot(dataX,columnX,dataY,columnY,dataZ='clim',columnZ='p_seasonality',gauge=[],colors=0,acceptable=[])` Plot of all gauges on x/y field, with color scale and gauge emphasis options

`fig=subsetID(dataX,columnX,xrange,dataY,columnY,yrange,dataZ='clim',columnZ='p_seasonality',zrange=[],colors=0,showthese=[],plot=1)` Narrow down the plot from the above

`fig=FDCstack(subset,ylims=[0.01,100],subsetname='Subset')` Returns overlayed scaled FDCs for a list of gauges

`fig=HydrographStack(subset,ylims=[0.001,10],subsetname='Subset')` Returns overlayed scaled median annual hydrographs for a list of gauges. `HydrographStackSLOW()` is the same but doesn't rely on pre-calculated table.
 
### [Forcing Data Plots](#Forcing-Data)
`data=loadforcing(gauge,dataset='daymet')` Loads forcing data (T, precip etc) from chosen dataset.
 
`fig=ForcingMinMedMax(gauge,columnname,dataset='daymet')` Displays plot of period of record min/med/max for forcing value over calendar year 

`fig=RasterPrecip(gauge,dataset='daymet')` Like the raster hydrograph, but for precipitation

`fig=RasterTemps(gauge,'max',dataset='daymet')` Like the raster hydrograph, but for min or max daily temperatures

`fig=RollingRasterPrecip(gauge,dataset='daymet',window=30)` Creates a raster plot of the n day sum of precipitation for each day

`fig=RollingTypicalPrecip(gauge,dataset='daymet',window=30)`  Gives period of record min/med/max for precipitation over year given a window length

`fig=RollingPrcp_select(gauge,selected_years,dataset='daymet',window=30)` As above, but with selected years on top

`fig=TypicalTemps(gauge,dataset='daymet')` Plots median daily min/max temperatures for each calendar day and absolute min/max temperatures over period of record

`fig=TypicalTemp_selectyear(gauge,selectedyear,dataset='daymet')` Plots a single year's daily T min/max values over the TypicalTemps plot 

### [Gauge Filtering](#Selected-Gauges-by-Attributes)
`acceptable=gaugefilter(subset,column,values)` Standalone, selects gauges within range of values for a field in one of the attribute/signature sets

`plotsquare(xrange,yrange,boxtype='k:',label=[])` Plots a box based on an x/y ranges on top of another plot

### [Other People's Stuff](#Things-other-people-made-that-are-useful)

`shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap')` Shifts a divergent colormap to be re-centered, say to 0


# Setting Home folder
In this folder, downloaded and expand the CAMELS datasets with the folders:
`/basin_timeseries_v1p2_metForcing_obsFlow/`
and 
`/camels_attributes_v2.0/`

In [1]:
#these are just what I have them as, edit them as needed to fit whatever folders you have
home = '/Users/DJHeins/Documents/Coding/CAMELS/' 
processed_data=home+ 'Scripts/CleanedScripts/GitHub/Data/'

# Setting Pre-requisite Functions / Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import math as m

def loadflow(gauge):
    """Returns dataframe directly loading the CAMELS provided streamflow data"""
    gaugeinfo = home + 'basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/basin_metadata/gauge_information.txt'
    allstreamflows = home + 'basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/usgs_streamflow/'
    outputs = home+'Scripts/Outputs/'

    gauges = pd.read_csv(gaugeinfo,sep='\t',header=0)  #had to adjust headers to line up right

    huc_id=gauges[gauges.GAGE_ID==gauge]['HUC_02'].values[0]

    GaugeInfo=gauges[gauges.GAGE_ID==gauge]

    if huc_id <10:
        nhuc = '0'+str(huc_id)
    else:
        nhuc = str(huc_id)

    streamflows = allstreamflows+nhuc+'/'

    if gauge < 10000000:
        data=streamflows+'0'+str(gauge)+'_streamflow_qc.txt'
    else:
        data=streamflows+str(gauge)+'_streamflow_qc.txt'

    data=pd.read_csv(data,sep='\s+',header=None)
    return data

def FDC(gauge):
    """Returns dataframe with FDC info - Flow, Count Exceeded, Percent Exceeded
    Dependent on loadflow(gauge)"""
    
    data=loadflow(gauge)

    # Remove any years missing a lot of data
    numdays=329 #minimum number of days
    fdc_data=data
    fdc_data=fdc_data[fdc_data[4] != -999.0] #remove nulls
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(fdc_data[1]==year)<numdays:
            fdc_data=fdc_data[fdc_data[1] != year]

    flows=fdc_data[4].values
    N=np.size(flows)
    P=100*np.append(np.linspace(0,1,100,endpoint=False),[.996,.997,.998,.999,.9999])
    threshold=np.percentile(flows,P)
    I=np.size(threshold)

    A=np.zeros((I,3))
    for i in range(0,I):
        A[i,0]=threshold[i]
        A[i,1]=np.sum(flows>threshold[i])
        A[i,2]=100*A[i,1]/N
    fdc=pd.DataFrame({'Flow':A[:,0],'Count Exceed':A[:,1],'Percent Exceed':A[:,2]})
    return fdc 

# FDC Plots

In [3]:
def FDC_breakpoints(gauge, scaled=1):
    """Finds optimal breakpoint for 2 line representation of FDC, plots it
    Dependent on loadflow(gauge)
    scaled=1 (default): scales to mean flow
    scaled=0: leaves flows in ft^3/s"""
    
    upslope_value=90
    downslope_value=10
    
    data=loadflow(gauge)
    
    # FLOW DURATION CURVE
    #removing partial years/nulls 
    numdays=329 #minimum number of days to be accepted
    data=data[data[4] != -999.0] #remove nulls
    yr1=min(data[1].values)
    yrEnd=max(data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(data[1]==year)<numdays:
            data=data[data[1] != year]

    flows=data[4].values

    #RUN DRY? 
    if np.min(flows) == 0:
        Dry=1
        PctFlow=np.sum(flows!=0)/np.size(flows)
        flows=data[data[4]!=0][4].values
    else:
        Dry=0
        PctFlow=100

    P=100*np.append(np.linspace(0,1,100,endpoint=False),[.995,.9975,.999,.9999])
    threshold=np.percentile(flows,P)
    if scaled==1: meanflow = np.mean(flows); modthresh=threshold/meanflow
    if scaled==0: modthresh=threshold
    fdc=pd.DataFrame({'Flow':threshold,'Scaled Flow':modthresh,'Percent Exceed':100-P})
    pct=fdc['Percent Exceed'].values
    flw=fdc['Scaled Flow'].values 

    #ONE LINE PREDICTOR
    upslope=flw[pct==upslope_value]
    downslope=flw[pct==downslope_value]

    m=(np.log(upslope)-np.log(downslope))/(downslope_value-upslope_value)
    def slopeprediction(x):
        return np.exp(np.log(upslope)-m*(x-upslope_value))

    pct_subset=pct[pct>downslope_value][pct[pct>downslope_value]<upslope_value]
    predictions_oneline=slopeprediction(pct_subset)
    flw_subset=flw[pct>downslope_value][pct[pct>downslope_value]<upslope_value]


    #BEST BREAKPOINT DETERMINATION
    breakpoint_tests=pct_subset 
    N_tests=np.size(breakpoint_tests)

    def slopeprediction_2(x):
        if x >= breakpoint:
            return np.exp(np.log(upslope)-m_A*(x-upslope_value))
        else:
            return np.exp(np.log(breakpoint_flw)-m_B*(x-breakpoint))

    out=np.zeros(shape=(N_tests,5))
    for index in range(0,N_tests):
        breakpoint=breakpoint_tests[index]
        breakpoint_flw=flw[pct==breakpoint]
        m_A=(np.log(upslope)-np.log(breakpoint_flw))/(breakpoint-upslope_value)
        m_B=(np.log(breakpoint_flw)-np.log(downslope))/(downslope_value-breakpoint)

        predictions_2=np.zeros(shape=np.shape(flw_subset))
        for i in range (0,np.size(flw_subset)):
            predictions_2[i]=slopeprediction_2(pct_subset[i])
        log_diff_2=np.log(predictions_2)-np.log(flw_subset)
        abs_log_diff_2=np.abs(log_diff_2)
        error=np.mean(abs_log_diff_2)
        error_b=np.mean(log_diff_2)

        out[index,0]=breakpoint
        out[index,1]=error
        out[index,2]=m_A
        out[index,3]=m_B
        out[index,4]=error_b
    out=pd.DataFrame(out,columns=['pct','err','m_A','m_B','err_b'])
    true_2line=out[out.err==np.min(out.err)]

    true_breakpoint=true_2line.pct.values
    meanabsdiff_2line=true_2line.err.values
    m_A=true_2line.m_A.values
    m_B=true_2line.m_B.values
    breakpoint_flw=flw[pct==true_breakpoint]

    #PREDICTED VALUES 2 LINES
    def slopeprediction_2line(x):
        if x >= true_breakpoint:
            return np.exp(np.log(upslope)-m_A*(x-upslope_value))
        else:
            return np.exp(np.log(breakpoint_flw)-m_B*(x-true_breakpoint))
    L=np.size(pct_subset)
    predictions_2line=np.zeros(shape=(L,1))
    for i in range (0,L):
        predictions_2line[i]=slopeprediction_2line(pct_subset[i])

    #PLOTTING
    if Dry == 1:   #rescaling the plot if the catchment runs dry
        true_breakpoint=PctFlow*true_breakpoint
        pct=PctFlow*pct
        pct_subset=PctFlow*pct_subset
        upslope_value=PctFlow*upslope_value
        downslope_value=PctFlow*downslope_value
        
    fig=plt.figure()
    plt.semilogy(pct,flw,'b-')
    plt.semilogy(pct_subset,predictions_oneline,'--')
    plt.semilogy(pct_subset,predictions_2line,'c--')
    plt.semilogy(upslope_value,upslope,'r*',downslope_value,downslope,'r*',true_breakpoint,breakpoint_flw,'r*')
    if scaled==1: plt.ylabel('Daily Average Flow / Period of Record Mean')
    if scaled==0: plt.ylabel('Daily Average Flow (ft$^3$/s)')
    plt.xlabel('Percentage of Time Flow Exceeded')
    plt.title('FDC with Optimal Breakpoint, Gauge '+str(gauge))
    axes = plt.gca()
    axes.set_xlim([-3,103])
    plt.show()
    return fig  
    
def MedianFDC(gauge):
    """Returns plot of each year's FDC, median of that, and regular FDC
    Dependent on loadflow(gauge) and FDC(gauge)"""

    data=loadflow(gauge)
    fdc=FDC(gauge)
    
    # Remove any years missing a lot of data
    numdays=329 #minimum number of days
    fdc_data=data
    fdc_data=fdc_data[fdc_data[4] != -999.0] #remove nulls
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(fdc_data[1]==year)<numdays:
            fdc_data=fdc_data[fdc_data[1] != year]

    # FLOW DURATION CURVE EVERY YEAR
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    years=np.unique(fdc_data[1].values)
    P=100*np.linspace(0,1,100,endpoint=False)
    I=np.size(P)
    nyears=np.size(years)
    A=np.zeros((nyears,I))

    fig=plt.figure()

    for i in range(0,nyears):
        year=years[i]
        tempdata=fdc_data[fdc_data[1] == year]
        flows=tempdata[4].values
        N=np.size(flows)
        P=100*np.linspace(0,1,100,endpoint=False)
        threshold=np.percentile(flows,P)
        A[i,:]=threshold
        plt.semilogy(100-P,threshold,'c-',alpha=0.3)

    medianFDC=np.median(A, axis=0)
    plt.semilogy([],[],'c-',label='Individual years')
    plt.semilogy(fdc['Percent Exceed'],fdc['Flow'],'b-',label='Period of record FDC')
    plt.semilogy(100-P,medianFDC,'r-',label='Median annual FDC')
    plt.title('Flow Duration Curve (Daily) at Gauge '+str(gauge))
    plt.ylabel('Daily Average Flow ($ft^3$/s)')
    plt.xlabel('Percentage of Time Flow Exceeded')
    plt.legend() #(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.show()
    return fig

def MedianScaledFDC(gauge):
    """Returns plot of each year's FDC scaled to annual means, median of that, and regular FDC
    Dependent on loadflow(gauge) and FDC(gauge)"""
    
    data=loadflow(gauge)
    fdc=FDC(gauge)
    
    # Remove any years missing a lot of data
    numdays=329 #minimum number of days
    fdc_data=data
    fdc_data=fdc_data[fdc_data[4] != -999.0] #remove nulls
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(fdc_data[1]==year)<numdays:
            fdc_data=fdc_data[fdc_data[1] != year]

    QBar=fdc_data.mean()[4]

    # FLOW DURATION CURVE EVERY YEAR
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    years=np.unique(fdc_data[1].values)
    P=100*np.linspace(0,1,100,endpoint=False)
    I=np.size(P)
    nyears=np.size(years)
    A=np.zeros((nyears,I))
    
    fig=plt.figure()

    for i in range(0,nyears):
        year=years[i]
        tempdata=fdc_data[fdc_data[1] == year]
        flows=tempdata[4].values
        
        qyear=np.mean(flows)
        flows=flows/qyear
        
        N=np.size(flows)
        P=100*np.linspace(0,1,100,endpoint=False)
        threshold=np.percentile(flows,P)
        A[i,:]=threshold
        plt.semilogy(100-P,threshold,'c-',alpha=0.3)

    medianFDC=np.median(A, axis=0)
    plt.semilogy([],[],'c-',label='Individual years')
    plt.semilogy(fdc['Percent Exceed'],fdc['Flow']/QBar,'b-',label='Period of record FDC')
    plt.semilogy(100-P,medianFDC,'r-',label='Median annual FDC')
    plt.title('Scaled Median FDC, Gauge '+str(gauge))
    plt.ylabel('Scaled Daily Average Flow')
    plt.xlabel('Percentage of Time Flow Exceeded')
    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.show()
    return fig 

def PeriodHydrographFDC(gauge):
    """Plots period of record FDC overlayed on Period Hydrograph
    Dependent on loadflow(gauge) and FDC(gauge)"""

    data=loadflow(gauge)
    fdc=FDC(gauge)
    
    # FDC vs Period Hydrograph
    fig, ax1 = plt.subplots()
    ax1.semilogy(data[4],'c-')
    ax1.set_ylabel('Daily Average Flow ($ft^3$/s)')
    ax2 = ax1.twiny()  # instantiate a second axes that shares the same x-axis
    ax2.semilogy(fdc['Percent Exceed'],fdc['Flow'],'b.-')
    ax1.tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False)
    ax2.tick_params(axis='x',which='both',bottom=True,top=False,labelbottom=True,labeltop=False) 
    plt.title('Flow Duration Curve (Daily) at Gauge '+str(gauge))
    #fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()
    return fig 

In [ ]:
def HydrographFDCOverlay(gauge):
    """Makes an overlay of the annual hydrograph range and FDC with 1 & 2 line approximations. Code is not cleaned up."""
    # FDC PART

    scaled=0
    upslope_value=90
    downslope_value=10

    data=loadflow(gauge)

    # FLOW DURATION CURVE
    #removing partial years/nulls 
    numdays=329 #minimum number of days to be accepted
    data=data[data[4] != -999.0] #remove nulls
    yr1=min(data[1].values)
    yrEnd=max(data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(data[1]==year)<numdays:
            data=data[data[1] != year]

    flows=data[4].values

    #RUN DRY? 
    if np.min(flows) == 0:
        Dry=1
        PctFlow=np.sum(flows!=0)/np.size(flows)
        flows=data[data[4]!=0][4].values
    else:
        Dry=0
        PctFlow=100

    P=100*np.append(np.linspace(0,1,100,endpoint=False),[.995,.9975,.999,.9999])
    threshold=np.percentile(flows,P)
    if scaled==1: meanflow = np.mean(flows); modthresh=threshold/meanflow
    if scaled==0: modthresh=threshold
    fdc=pd.DataFrame({'Flow':threshold,'Scaled Flow':modthresh,'Percent Exceed':100-P})
    pct=fdc['Percent Exceed'].values
    flw=fdc['Scaled Flow'].values 

    #ONE LINE PREDICTOR
    upslope=flw[pct==upslope_value]
    downslope=flw[pct==downslope_value]

    m=(np.log(upslope)-np.log(downslope))/(downslope_value-upslope_value)
    def slopeprediction(x):
        return np.exp(np.log(upslope)-m*(x-upslope_value))

    pct_subset=pct[pct>downslope_value][pct[pct>downslope_value]<upslope_value]
    predictions_oneline=slopeprediction(pct_subset)
    flw_subset=flw[pct>downslope_value][pct[pct>downslope_value]<upslope_value]

    #BEST BREAKPOINT DETERMINATION
    breakpoint_tests=pct_subset 
    N_tests=np.size(breakpoint_tests)

    def slopeprediction_2(x):
        if x >= breakpoint:
            return np.exp(np.log(upslope)-m_A*(x-upslope_value))
        else:
            return np.exp(np.log(breakpoint_flw)-m_B*(x-breakpoint))

    out=np.zeros(shape=(N_tests,5))
    for index in range(0,N_tests):
        breakpoint=breakpoint_tests[index]
        breakpoint_flw=flw[pct==breakpoint]
        m_A=(np.log(upslope)-np.log(breakpoint_flw))/(breakpoint-upslope_value)
        m_B=(np.log(breakpoint_flw)-np.log(downslope))/(downslope_value-breakpoint)

        predictions_2=np.zeros(shape=np.shape(flw_subset))
        for i in range (0,np.size(flw_subset)):
            predictions_2[i]=slopeprediction_2(pct_subset[i])
        log_diff_2=np.log(predictions_2)-np.log(flw_subset)
        abs_log_diff_2=np.abs(log_diff_2)
        error=np.mean(abs_log_diff_2)
        error_b=np.mean(log_diff_2)

        out[index,0]=breakpoint
        out[index,1]=error
        out[index,2]=m_A
        out[index,3]=m_B
        out[index,4]=error_b
    out=pd.DataFrame(out,columns=['pct','err','m_A','m_B','err_b'])
    true_2line=out[out.err==np.min(out.err)]

    true_breakpoint=true_2line.pct.values
    meanabsdiff_2line=true_2line.err.values
    m_A=true_2line.m_A.values
    m_B=true_2line.m_B.values
    breakpoint_flw=flw[pct==true_breakpoint]

    #PREDICTED VALUES 2 LINES
    def slopeprediction_2line(x):
        if x >= true_breakpoint:
            return np.exp(np.log(upslope)-m_A*(x-upslope_value))
        else:
            return np.exp(np.log(breakpoint_flw)-m_B*(x-true_breakpoint))
    L=np.size(pct_subset)
    predictions_2line=np.zeros(shape=(L,1))
    for i in range (0,L):
        predictions_2line[i]=slopeprediction_2line(pct_subset[i])

    #PLOTTING
    if Dry == 1:   #rescaling the plot if the catchment runs dry
        true_breakpoint=PctFlow*true_breakpoint
        pct=PctFlow*pct
        pct_subset=PctFlow*pct_subset
        upslope_value=PctFlow*upslope_value
        downslope_value=PctFlow*downslope_value
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # HYDROGRAPH PART
    data=loadflow(gauge)
    data=data.replace(-999.0,np.nan)

    yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # PLOTTING PART            

    fig, ax1 = plt.subplots()

    ax1.semilogy(yrsum['Max'],'k:',alpha=0.3,label='Max at Date')
    ax1.semilogy(yrsum['Median'],'k-',alpha=0.3,label='Median at Date')
    ax1.semilogy(yrsum['Min'],'k:',alpha=0.3,label='Min at Date')
    plt.legend(bbox_to_anchor=(1.02, 0.75), loc=2, borderaxespad=0.)
    #plt.title('Annual Variations in Streamflow at Gauge '+str(gauge))
    ax1.set_ylabel('Flow ($ft^3$/s)')
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)

    ax2 = ax1.twiny()  # instantiate a second axes that shares the same x-axis
    ax2.semilogy(pct,flw,'b-',label='FDC')
    ax2.semilogy(pct_subset,predictions_oneline,'--',label='90-10 Straight Line')
    ax2.semilogy(pct_subset,predictions_2line,'c--',label='2 Line Approximation')
    ax2.semilogy(upslope_value,upslope,'r*',downslope_value,downslope,'r*',true_breakpoint,breakpoint_flw,'r*')
    if scaled==1: plt.ylabel('Daily Average Flow / Period of Record Mean')
    #if scaled==0: plt.ylabel('Daily Average Flow (ft$^3$/s)')
    plt.xlabel('Percentage of Time Flow Exceeded')
    #plt.title('FDC with Optimal Breakpoint, Gauge '+str(gauge))
    #axes = plt.gca()
    ax2.set_xlim([-3,103])
    plt.title('FDC & Typical Hydrograph at Gauge '+str(gauge), pad=35)
    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.show()


# Other Streamflow Plots

In [ ]:
def RasterHydrograph(gauge):
    """Return image of raster hydrograph, with no data in black
    Dependent on loadflow(gauge)"""
    
    data=loadflow(gauge)
    data=data.replace(-999.0,np.nan)
    if data[2][0] != 1 | data[3][0] != 1:  #if a data doesn't start at Jan 1st, toss that year
        display('Note: raster hydrograph removed incomplete year '+str(data[1][0]))
        data=data[data[1] != data[1][0]]
        
    Years=data[1].unique()
    NYears=np.size(Years)
    RastHydr = np.empty([NYears,365])
    StartYear=Years[0]
    EndYear=Years[NYears-1]

    for nyear in range(0,NYears):
        yrdata=data[data[1]==Years[nyear]][4].values
        if np.size(yrdata)==366: #leap years are a pain, let's just delete feb 29 from all years 
            yrdata=np.delete(yrdata,59)
        if np.size(yrdata)==365:
            RastHydr[nyear,:]=yrdata
        else:
            RastHydr[nyear,:]=np.full((1,365),np.nan)
            display('Note: raster hydrograph removed incomplete year '+str(Years[nyear]))

    RHscaled=np.log10(RastHydr+0.001) #making 0 flow days not break the log scale

    fig=plt.figure()

    Blues2=plt.cm.Blues
    Blues2.set_bad('black',1.)

    imgplot=plt.imshow(RHscaled,interpolation='nearest', aspect='auto') #,extent=[0,25,0,20])
    imgplot.set_cmap(Blues2)
    plt.title('Daily Flow at Gauge '+str(gauge)+', '+str(StartYear)+'-'+str(EndYear))
    plt.colorbar(label='log10(daily average flow in  $ft^3$/s)')
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='major',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True) # labels along the bottom edge are off

    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    plt.xticks([15,45,74,105,135,166,196,227,258,288,319,349], xticklabels)
    labelnum=range(0,NYears,2)
    yticklabels=np.ones(np.size(labelnum),dtype=np.int16)
    for n in range(0,np.size(labelnum)):
        yticklabels[n]=StartYear+labelnum[n]
    plt.yticks(range(0,NYears,2), yticklabels)
    plt.show()
    return fig 

def StackedHydrograph(gauge):
    """Returns plot of each year's hydrograph year with min/med/max values for each day
    Dependent on loadflow(gauge)"""

    data=loadflow(gauge)
    data=data.replace(-999.0,np.nan)

    # Remove any years missing a lot of data
    numdays=329 #minimum number of days
    fdc_data=data
    fdc_data=fdc_data[fdc_data[4] != -999.0] #remove nulls
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(fdc_data[1]==year)<numdays:
            fdc_data=fdc_data[fdc_data[1] != year]

    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    years=np.unique(fdc_data[1].values)
    P=100*np.linspace(0,1,100,endpoint=False)
    I=np.size(P)
    nyears=np.size(years)
    A=np.zeros((nyears,I))

    yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
    fig=plt.figure()

    fig, ax1 = plt.subplots()

    for i in range(0,nyears):
        year=years[i]
        tempdata=fdc_data[fdc_data[1] == year]
        flows=tempdata[4].values
        plt.semilogy(flows,'c-',alpha=0.3)
    ax1.semilogy(yrsum['Max'],'b-',label='Max',alpha=0.5)
    ax1.semilogy(yrsum['Median'],'k-',label='Median',alpha=0.7)
    ax1.semilogy(yrsum['Min'],'r-',label='Min',alpha=0.5)
    ax1.semilogy([],'c-',label='Individual Years',alpha=0.3)
    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title('Annual Variations in Streamflow at Gauge '+str(gauge))
    ax1.set_ylabel('Flow ($ft^3$/s)')
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig

def TypicalYear(gauge):
    """Returns plot of typical year with min/med/max values for each day
    Dependent on loadflow(gauge)"""
   
    data=loadflow(gauge)
    data=data.replace(-999.0,np.nan)

    yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1

    fig, ax1 = plt.subplots()
    
    ax1.semilogy(yrsum['Max'],'b-',label='Max')
    ax1.semilogy(yrsum['Median'],'k-',label='Median')
    ax1.semilogy(yrsum['Min'],'r-',label='Min')
    plt.legend() #(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title('Annual Variations in Streamflow at Gauge '+str(gauge))
    ax1.set_ylabel('Flow ($ft^3$/s)')
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig 

def multiplot(gauge):
    """plots MapMaker,TypicalYear,RasterHydrograph, & PeriodHydrographFDC for a gauge"""
    fig=MapMaker(gauge)
    fig=TypicalYear(gauge)
    fig=RasterHydrograph(gauge)
    fig=PeriodHydrographFDC(gauge)

# I'M THE MAP!

In [1]:
def CONUSplot():
    """Makes a plot of the continental US, can be plotted over""" 
    
    #add needed libraries
    import matplotlib.patches as mpatches
    import shapely.geometry as sgeom
    import cartopy.crs as ccrs
    import cartopy.io.shapereader as shpreader
    
    #set domain
    ax = plt.axes([0.01, 0.01, 0.98, 0.98], projection=ccrs.PlateCarree())
    ax.set_xlim([-125, -66.5]); ax.set_ylim([23, 50])
    
    #load shapefiles
    shapename = 'admin_1_states_provinces_lakes_shp'
    states_shp = shpreader.natural_earth(resolution='110m', category='cultural', name=shapename)
    
    # turn off the outline and background patches
    ax.background_patch.set_visible(False)
    ax.outline_patch.set_visible(False)
    
    #plot each state
    for state in shpreader.Reader(states_shp).geometries():
        facecolor = [0.95, 0.95, 0.95]
        edgecolor = 'grey'
        ax.add_geometries([state], ccrs.PlateCarree(),facecolor=facecolor, edgecolor=edgecolor)
        
def MapMaker(gauge):
    """Returns map of CONUS with red dot at gauge location
    dependent on CONUSplot()"""
    
    gaugeinfo = home + 'basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/basin_metadata/gauge_information.txt'

    gauges = pd.read_csv(gaugeinfo,sep='\t',header=0)  #had to adjust headers to line up right
    huc_id=gauges[gauges.GAGE_ID==gauge]['HUC_02'].values[0]

    GaugeInfo=gauges[gauges.GAGE_ID==gauge]

    [lat,lon]=[GaugeInfo['LAT'].values,GaugeInfo['LONG'].values] #identifying gauge coordinates
    fig=plt.figure()
    CONUSplot() #add plot of USA
    plt.plot(lon,lat,'ro')
    plt.title('Gauge '+str(gauge))

    plt.show()
    return fig 

def mapdata(category,column,colorscale='divergent',gauge=[],subset=[],show=1,figon=1,size=15):
    """Plots map with colorscaled dots
    category:  'name','geol','clim','hydro','soil','topo','vege','OtherAnalysis','fdc_info' 
    column: column name from above dataset
    gauge: Can highlight individual gauge on map, optional/off by default
    subset: Can narrow to subset of gauges with a *list* of gauge IDs
    show: 1 if make a figure as is, 0 if want to modify it
    colorscale: decide color system
        'linear', 'divergent', or 'custom'
        if custom: col, shiftedcmap = customcolors() ...must be defined prior """
    fdc_infoALL=returnattributeset('fdc_info')
    if subset==[]:   subset=fdc_infoALL.gauge
    category=returnattributeset(category)
    df1=fdc_infoALL[fdc_infoALL.gauge.isin(subset)][['gauge','LAT','LONG']]
    df2=category[fdc_infoALL.gauge.isin(subset)][[column]]
    df1=pd.concat([df1, df2], axis=1)

    gaugeinfo = home + 'basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/basin_metadata/gauge_information.txt'

    if figon==1: fig=plt.figure(figsize=(8,6))
    CONUSplot()

    col=df1[column]
    if colorscale=='divergent':
        colormap=plt.cm.coolwarm_r
        colorscaledata=col
        cmapmidpoint=1-max(colorscaledata)/(max(colorscaledata)+abs(min(colorscaledata)))
        shiftedcmap=shiftedColorMap(colormap, start=0, midpoint=cmapmidpoint, stop=1, name='shiftedcmap')
    if colorscale=='linear':
        shiftedcmap= plt.cm.viridis_r #Blues #inferno_r #
    if colorscale=='PctFlow':
        subsetZ_B=returnattributeset('fdc_info')
        col=subsetZ_B['PctFlow']
        colormap= plt.cm.plasma_r
        colorscaledata=col
        shiftedcmap=shiftedColorMap(colormap, start=-0.2, midpoint=.98, stop=1, name='shiftedcmap')
        colorbarlabel='Percentage of days with flow'

    plt.set_cmap(shiftedcmap)
    plt.scatter(df1['LONG'],df1['LAT'],zorder=2,c=col,s=size,edgecolor='k',linewidths=0.25)  #zorder apparently puts it on top
    plt.colorbar(shrink=0.4,aspect=15,label=column)
    if gauge!=[]:
        details=df1[df1['gauge']==gauge]
        plt.scatter(details['LONG'],details['LAT'],zorder=3,s=40,facecolors='none',edgecolor='k',linewidth=1.5)
    if show==1: 
        plt.show()
        return fig

# Plots to Emphasize Selected Years

In [3]:
def MedianFDC_selectyears(gauge,selected_years):
    """Returns plot specific year FDCs over all FDCs, median FDC, and regular FDC 
    Dependent on loadflow(gauge) and FDC(gauge)"""

    data=loadflow(gauge)
    fdc=FDC(gauge)
    
    # Remove any years missing a lot of data
    numdays=329 #minimum number of days
    fdc_data=data
    fdc_data=fdc_data[fdc_data[4] != -999.0] #remove nulls
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(fdc_data[1]==year)<numdays:
            fdc_data=fdc_data[fdc_data[1] != year]

    # FLOW DURATION CURVE EVERY YEAR
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    years=np.unique(fdc_data[1].values)
    P=100*np.linspace(0,1,100,endpoint=False)
    I=np.size(P)
    nyears=np.size(years)
    A=np.zeros((nyears,I))

    fig=plt.figure()

    for i in range(0,nyears):
        year=years[i]
        tempdata=fdc_data[fdc_data[1] == year]
        flows=tempdata[4].values
        N=np.size(flows)
        P=100*np.linspace(0,1,100,endpoint=False)
        threshold=np.percentile(flows,P)
        A[i,:]=threshold
        plt.semilogy(100-P,threshold,'c-',alpha=0.2)

    medianFDC=np.median(A, axis=0)
    plt.semilogy([],[],'c-',label='Individual years')
    plt.semilogy(fdc['Percent Exceed'],fdc['Flow'],'b-',label='Period of record FDC',alpha=0.5)
    plt.semilogy(100-P,medianFDC,'r-',label='Median annual FDC',alpha=0.5)

    for year in selected_years:
        tempdata=fdc_data[fdc_data[1] == year]
        flows=tempdata[4].values
        N=np.size(flows)
        P=100*np.linspace(0,1,100,endpoint=False)
        threshold=np.percentile(flows,P)
        A[i,:]=threshold
        plt.semilogy(100-P,threshold,'-',linewidth=2,label=str(year))
        
    plt.title('Flow Duration Curve (Daily) at Gauge '+str(gauge))
    plt.ylabel('Daily Average Flow ($ft^3$/s)')
    plt.xlabel('Percentage of Time Flow Exceeded')
    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.show()
    
    return fig

def MedianScaledFDC_selectyears(gauge,selected_years):
    """Returns plot specific year scaled FDCs over all scaled FDCs, median FDC, and regular FDC 
    Dependent on loadflow(gauge) and FDC(gauge)"""
    
    data=loadflow(gauge)
    fdc=FDC(gauge)
    
    # Remove any years missing a lot of data
    numdays=329 #minimum number of days
    fdc_data=data
    fdc_data=fdc_data[fdc_data[4] != -999.0] #remove nulls
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    for year in range(yr1,yrEnd+1):
        if np.sum(fdc_data[1]==year)<numdays:
            fdc_data=fdc_data[fdc_data[1] != year]

    QBar=fdc_data.mean()[4]

    # FLOW DURATION CURVE EVERY YEAR
    yr1=min(fdc_data[1].values)
    yrEnd=max(fdc_data[1].values)
    years=np.unique(fdc_data[1].values)
    P=100*np.linspace(0,1,100,endpoint=False)
    I=np.size(P)
    nyears=np.size(years)
    A=np.zeros((nyears,I))
    
    fig=plt.figure()

    for i in range(0,nyears):
        year=years[i]
        tempdata=fdc_data[fdc_data[1] == year]
        flows=tempdata[4].values
        
        qyear=np.mean(flows)
        flows=flows/qyear
        
        N=np.size(flows)
        P=100*np.linspace(0,1,100,endpoint=False)
        threshold=np.percentile(flows,P)
        A[i,:]=threshold
        plt.semilogy(100-P,threshold,'c-',alpha=0.2)

    medianFDC=np.median(A, axis=0)
    plt.semilogy([],[],'c-',label='Individual years')
    plt.semilogy(fdc['Percent Exceed'],fdc['Flow']/QBar,'b-',label='Period of record FDC',alpha=0.5)
    plt.semilogy(100-P,medianFDC,'r-',label='Median annual FDC',alpha=0.5)
    
    for year in selected_years:
        tempdata=fdc_data[fdc_data[1] == year]
        flows=tempdata[4].values
        qyear=np.mean(flows)
        flows=flows/qyear        
        N=np.size(flows)
        P=100*np.linspace(0,1,100,endpoint=False)
        threshold=np.percentile(flows,P)
        A[i,:]=threshold
        plt.semilogy(100-P,threshold,'-',linewidth=2,label=str(year))
        
    plt.title('Scaled Median FDC, Gauge '+str(gauge))
    plt.ylabel('Scaled Daily Average Flow')
    plt.xlabel('Percentage of Time Flow Exceeded')
    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.show()
    return fig 

def TypicalYear_selectyears(gauge,selected_years):
    """Returns hydrograph of chosen years over period of record min/med/max 
    Dependent on loadflow(gauge)"""

    data=loadflow(gauge)
    data=data.replace(-999.0,np.nan)

    yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                yrsum.loc[n,'Min']=data[(data[3]==day) & (data[2]==month)][4].min()
                yrsum.loc[n,'Max']=data[(data[3]==day) & (data[2]==month)][4].max()
                n=n+1

    fig, ax1 = plt.subplots()

    ax1.semilogy(yrsum['Max'],'b-',label='Max',alpha=0.2)
    ax1.semilogy(yrsum['Median'],'k-',label='Median',alpha=0.2)
    ax1.semilogy(yrsum['Min'],'r-',label='Min',alpha=0.2)

    for year in selected_years:
        tempdata=data[data[1] == year]
        ax1.semilogy(range(0,365),tempdata[4],'-',label=str(year))

    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title('Annual Variations in Streamflow at Gauge '+str(gauge))


    ax1.set_ylabel('Flow ($ft^3$/s)')
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig 

# Returning Attribute Data

In [5]:
def returnattributes(gauge):
    """Displays all CAMELS provided attributes for a gauge, along with two sets of calculated streamflow figures"""
    
    folder=home+'camels_attributes_v2.0/'; 
    atttype='name'; filename= folder+'camels_'+atttype+'.txt'; name_ALL= pd.read_csv(filename,sep=';',header=0); name=name_ALL[name_ALL.gauge_id==gauge]; 
    atttype='geol'; filename= folder+'camels_'+atttype+'.txt'; geol_ALL= pd.read_csv(filename,sep=';',header=0); geol=geol_ALL[geol_ALL.gauge_id==gauge]; 
    atttype='clim'; filename= folder+'camels_'+atttype+'.txt'; clim_ALL= pd.read_csv(filename,sep=';',header=0); clim=clim_ALL[clim_ALL.gauge_id==gauge]; 
    atttype='hydro'; filename= folder+'camels_'+atttype+'.txt'; hydro_ALL= pd.read_csv(filename,sep=';',header=0); hydro=hydro_ALL[hydro_ALL.gauge_id==gauge]; 
    atttype='soil'; filename= folder+'camels_'+atttype+'.txt'; soil_ALL= pd.read_csv(filename,sep=';',header=0); soil=soil_ALL[soil_ALL.gauge_id==gauge]; 
    atttype='topo'; filename= folder+'camels_'+atttype+'.txt'; topo_ALL= pd.read_csv(filename,sep=';',header=0); topo=topo_ALL[topo_ALL.gauge_id==gauge]; 
    atttype='vege'; filename= folder+'camels_'+atttype+'.txt'; vege_ALL= pd.read_csv(filename,sep=';',header=0); vege=vege_ALL[vege_ALL.gauge_id==gauge]; 

    filepath=processed_data+'GaugeSummaryV6.txt'; 
    OtherAnalysis= pd.read_csv(filepath,sep='\t',header=0); 
    OtherAnalysis_ALL=OtherAnalysis[['Gauge','HUC2','Years','Mean','Std','sd_Qann-QBar','sd_Qmon-Qann','sd_Qday-Qmon','Med','fdc20','fdc80','AREA_(KM^2)']]; 
    OtherAnalysis=OtherAnalysis_ALL[OtherAnalysis_ALL.Gauge==gauge]; 

    filepath=processed_data+'FDC_Slope_BrkPt_Dry2.txt'; 
    fdc_infoALL=pd.read_csv(filepath,sep='\t',header=0); 
    fdc_info=fdc_infoALL[fdc_infoALL.gauge==gauge];
    
    display(fdc_info); print('FDC INFO'); 
    display(name); print('NAME'); display(OtherAnalysis); print('FLOWS INFO'); display(hydro); print('HYDROLOGY'); 
    display(clim); print('CLIMATE'); display(geol); print('GEOLOGY'); display(soil); print('SOIL'); 
    display(topo); print('TOPOGRAPHY'); display(vege); print('VEGETATION');
    
def returnattributeset(dataset): 
    """returns full dataframe of attributes/signatures
    dataset options: 
      'name','geol','clim','hydro','soil','topo','vege'
      'OtherAnalysis','fdc_info' 
    """
    folder=home+'camels_attributes_v2.0/'; 
    if dataset=='name':
        atttype='name'; filename= folder+'camels_'+atttype+'.txt'; df= pd.read_csv(filename,sep=';',header=0); 
    if dataset=='geol':
        atttype='geol'; filename= folder+'camels_'+atttype+'.txt'; df= pd.read_csv(filename,sep=';',header=0); 
    if dataset=='clim':
        atttype='clim'; filename= folder+'camels_'+atttype+'.txt'; df= pd.read_csv(filename,sep=';',header=0); 
    if dataset=='hydro':
        atttype='hydro';filename= folder+'camels_'+atttype+'.txt'; df= pd.read_csv(filename,sep=';',header=0); 
    if dataset=='soil':
        atttype='soil'; filename= folder+'camels_'+atttype+'.txt'; df= pd.read_csv(filename,sep=';',header=0); 
    if dataset=='topo':
        atttype='topo'; filename= folder+'camels_'+atttype+'.txt'; df= pd.read_csv(filename,sep=';',header=0); 
    if dataset=='vege':
        atttype='vege'; filename= folder+'camels_'+atttype+'.txt'; df= pd.read_csv(filename,sep=';',header=0); 
    if dataset=='OtherAnalysis':
        filepath=processed_data+'GaugeSummaryV6.txt'; OtherAnalysis= pd.read_csv(filepath,sep='\t',header=0); df=OtherAnalysis[['Gauge','HUC2','Years','Mean','Std','sd_Qann-QBar','sd_Qmon-Qann','sd_Qday-Qmon','Med','fdc20','fdc80','AREA_(KM^2)']]; 
    if dataset=='fdc_info':
        filepath=processed_data+'FDC_Slope_BrkPt_Dry2.txt'; df=pd.read_csv(filepath,sep='\t',header=0); 
    return df

def fieldlisting(dataX):
    """Returns list of numeric fields within each dataset"""
    
    if dataX== 'clim':
        columns=['p_mean','pet_mean','p_seasonality','frac_snow','aridity','high_prec_freq','high_prec_dur','low_prec_freq','low_prec_dur']

    if dataX=='topo':
        columns=['elev_mean','slope_mean','area_gages2','area_geospa_fabric']

    if dataX=='geol':
        columns=['carbonate_rocks_frac','geol_porostiy','geol_permeability']

    if dataX=='soil':
        columns=['soil_depth_pelletier','soil_depth_statsgo','soil_porosity','soil_conductivity','max_water_content','sand_frac','silt_frac','clay_frac','water_frac','organic_frac','other_frac']

    if dataX=='vege':
        columns=['frac_forest','lai_max','lai_diff','gvf_max','gvf_diff']

    if dataX=='hydro':
        columns=['q_mean','runoff_ratio','slope_fdc','baseflow_index','stream_elas','q5','q95','high_q_freq','high_q_dur','low_q_freq','low_q_dur','zero_q_freq','hfd_mean']

    if dataX=='fdc_info':
        columns=['PctFlow','m_90-10','err_1_abs','err_1b','break','err_2_abs','err_2b','m_1','m_2','err_ratio_abs']

    if dataX=='OtherAnalysis':
        columns=['Mean','Std','sd_Qann-QBar','sd_Qmon-Qann','sd_Qday-Qmon']
    return columns

# Some Comparison Plots 

In [ ]:
def comparisonplot(dataX,columnX,dataY,columnY,dataZ='clim',columnZ='p_seasonality',gauge=[],colors=0,s=10,acceptable=[],linewidths=0.25,show=1,alpha=0.7):
    """Return color scaled scatterplot comparing 2 or 3 fields
    dataX/Y/Z:  'name','geol','clim','hydro','soil','topo','vege','OtherAnalysis','fdc_info' 
    fieldX/Y/Z: column name from above dataset
        Z optional
    gauge: Can highlight individual gauge on plot
    acceptable: Can narrow to subset of gauges with a list of gauge ID
    show: 1 if make a figure as is, 0 if want to modify it
    colors: decide color system
        0: Grey dots, no colormap
        1: Binary of always flows/can run dry
        2: Pct Flow
        3: Divergent
        4: Linear
        5: col, shiftedcmap, colorbarlabel= customcolors() ...must be defined prior"""
    subsetX=returnattributeset(dataX)
    subsetY=returnattributeset(dataY)
    subsetZ=returnattributeset(dataZ)

    fdc_infoALL=returnattributeset('fdc_info')
    subsetZ_B=fdc_infoALL

    if acceptable!=[]:
        subsetX=subsetX[subsetX.iloc[:,0].isin(acceptable)]; 
        subsetY=subsetY[subsetY.iloc[:,0].isin(acceptable)]; 
        subsetZ=subsetZ[subsetZ.iloc[:,0].isin(acceptable)]; 
        subsetZ_B=subsetZ_B[subsetZ_B.iloc[:,0].isin(acceptable)]
    x=subsetX[columnX]; y=subsetY[columnY]

    if colors==1:
        col=subsetZ_B['Dry'] #'PctFlow']
        shiftedcmap= plt.cm.bwr #plasma_r
        colorbarlabel='Red: Runs Dry. Blue: Always flows'
    if colors==2:
        col=subsetZ_B['PctFlow']
        colormap= plt.cm.plasma_r
        colorscaledata=col
        shiftedcmap=shiftedColorMap(colormap, start=-0.2, midpoint=.96, stop=1, name='shiftedcmap')
        colorbarlabel='Percentage of days with flow'
    if colors==3:
        col=subsetZ[columnZ]
        colormap=plt.cm.coolwarm_r
        colorscaledata=col
        cmapmidpoint=1-max(colorscaledata)/(max(colorscaledata)+abs(min(colorscaledata)))
        shiftedcmap=shiftedColorMap(colormap, start=0, midpoint=cmapmidpoint, stop=1, name='shiftedcmap')
        colorbarlabel=columnZ
    if colors==4:
        col=subsetZ[columnZ]
        shiftedcmap= plt.cm.viridis_r #plasma_r 
        colorbarlabel=columnZ
    if colors ==5:
        col, shiftedcmap, colorbarlabel= customcolors() 

    fig=plt.figure()

    if colors!=0:    
        plt.set_cmap(shiftedcmap)
        plt.scatter(x,y,c=col,s=s,alpha=alpha,edgecolor='k',linewidths=linewidths)
        plt.colorbar(label=colorbarlabel);
    if colors==0:    plt.scatter(x,y,s=s,alpha=0.3,facecolors='k')
    plt.xlabel(columnX); plt.ylabel(columnY)
    if gauge!=[]:    plt.scatter(subsetX[subsetX.iloc[:,0]==gauge][columnX],subsetY[subsetY.iloc[:,0]==gauge][columnY],s=40,alpha=1,facecolors='none',edgecolor='k',linewidths=2)
    if show==1:      plt.show()
    return fig

def subsetID(dataX,columnX,xrange,dataY,columnY,yrange,dataZ='clim',columnZ='p_seasonality',zrange=[],colors=0,showthese=[],plot=1,primaryfilter=[]):
    """Return color scaled scatterplot comparing 2 or 3 fields
    dataX/Y/Z:  'name','geol','clim','hydro','soil','topo','vege','OtherAnalysis','fdc_info' 
    fieldX/Y/Z: column name from above dataset
        Z optional
    x/y/zrange: bracketed range of values for output to be within
    show: 1 if make a figure as is, 0 if want to modify it
    primaryfilter: optional, list of existing gauges that this should create a subset of
    colors: decide color system
        0/1/2: Grey dots, no colormap
        3: Divergent
        4: Linear
        5: col, shiftedcmap, colorbarlabel= customcolors() ...must be defined prior"""

    acceptable=gaugefilter(dataX,columnX,xrange) & gaugefilter(dataY,columnY,yrange)
    if zrange!=[]: acceptable = acceptable & gaugefilter(dataZ,columnZ,zrange)

    if primaryfilter !=[]:
        acceptable = acceptable & set(primaryfilter)
        
    dfX=returnattributeset(dataX)
    dfY=returnattributeset(dataY)
    dfZ=returnattributeset(dataZ)
    xgood=dfX[dfX.iloc[:,0].isin(acceptable)]
    ygood=dfY[dfY.iloc[:,0].isin(acceptable)]
    zgood=dfZ[dfZ.iloc[:,0].isin(acceptable)]

    if colors==1:    colors=0
    if colors==2:    colors=0
    if colors==3:
        col=zgood[columnZ]
        colormap=plt.cm.coolwarm_r
        colorscaledata=col
        cmapmidpoint=1-max(colorscaledata)/(max(colorscaledata)+abs(min(colorscaledata)))
        shiftedcmap=shiftedColorMap(colormap, start=0, midpoint=cmapmidpoint, stop=1, name='shiftedcmap')
        colorbarlabel=columnZ
    if colors==4:
        col=zgood[columnZ]
        shiftedcmap= plt.cm.viridis_r
        colorbarlabel=columnZ
    if colors ==5:
        col, shiftedcmap, colorbarlabel= customcolors() 

    fig=plt.figure()
    if colors!=0: 
        plt.scatter(xgood[columnX],ygood[columnY],c=col,s=20,alpha=0.6,edgecolor='k',linewidths=0.5)
        plt.colorbar(label=colorbarlabel); 
    if colors==0:
        plt.scatter(xgood[columnX],ygood[columnY],s=20,alpha=0.6,facecolors='b',edgecolor='k',linewidths=0.5)
    plt.xlabel(columnX); plt.ylabel(columnY)
    if plot==1:  plt.show()
    if 'x' in showthese:    display(xgood)
    if 'y' in showthese:    display(ygood)
    if 'z' in showthese:    display(zgood)
    return fig

def FDCstack(subset,ylims=[0.001,500],subsetname='Subset',alpha=0.2,color='k',show=1):
    """Returns overlayed scaled FDCs for a list of gauges
    Runs fairly slowly
    Dependent on loadflow(gauge) and FDC(gauge)"""
    
    subset=list(subset)
    ngauges=np.size(subset)
    fig=plt.figure()

    for gauge in subset:
        data = loadflow(gauge)
        fdc = FDC(gauge)

        # Remove any years missing a lot of data
        numdays=329 #minimum number of days
        fdc_data=data
        fdc_data=fdc_data[fdc_data[4] != -999.0] #remove nulls
        yr1=min(fdc_data[1].values)
        yrEnd=max(fdc_data[1].values)
        for year in range(yr1,yrEnd+1):
            if np.sum(fdc_data[1]==year)<numdays:
                fdc_data=fdc_data[fdc_data[1] != year]

        QBar=fdc_data.mean()[4]

        plt.semilogy(fdc['Percent Exceed'],fdc['Flow']/QBar,color=color,alpha=alpha)

    axes = plt.gca()
    axes.set_xlim([-3,103])
    axes.set_ylim(ylims)

    plt.ylabel('Daily Average Flow / Period of Record Mean')
    plt.xlabel('Percentage of Time Flow Exceeded') 
    plt.title('Scaled FDCs for '+str(ngauges)+' Gauges in '+subsetname)
    if show==1: plt.show()
    return fig

def HydrographStack(subset,ylims=[0.001,10],subsetname='Subset',alpha=0.4,color='k',show=1):
    """Returns overlayed scaled typical (mean) hydrographs for a list of gauges
    Runs fast, but requires MedAnnHydrograph.txt
    Dependent on loadflow(gauge) and FDC(gauge)"""
    
    filename=processed_data+'MedAnnHydrograph.txt'
    df = pd.read_csv(filename,sep='\t',header=0)
    
    subset=list(subset)
    ngauges=np.size(subset)
    if show==1:
        fig, ax1 = plt.subplots()
   
    for gauge in subset:
        ax1.semilogy(df[str(gauge)],color=color,alpha=alpha)

    #plt.title('Typical Hydrographs for '+str(ngauges)+' Gauges in '+subsetname)
    plt.title(str(ngauges)+' Gauges in '+subsetname)
    ax1.set_ylabel('Median Daily Flow/Period Mean Flow')
    axes = plt.gca()
    axes.set_ylim(ylims)
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    if show==1: plt.show()
    return fig

def HydrographStackSLOW(subset,ylims=[0.001,10],subsetname='Subset'):
    """Returns overlayed scaled typical (mean) hydrographs for a list of gauges
    Runs very slowly
    Dependent on loadflow(gauge) and FDC(gauge)"""
    
    subset=list(subset)
    ngauges=np.size(subset)
    fig, ax1 = plt.subplots()

    for gauge in subset:
        data = loadflow(gauge)
        data=data.replace(-999.0,np.nan)
  
        QBar=data.mean()[4]
        
        yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
        n=0
        for month in range(1,13,1):
            if month in [9,4,6,11]: #30 
                for day in range(1,31):
                    yrsum.loc[n,'Month']=month
                    yrsum.loc[n,'Day']=day
                    yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                    n=n+1
            if month in [1,3,5,7,8,10,12]: #31
                for day in range(1,32):
                    yrsum.loc[n,'Month']=month
                    yrsum.loc[n,'Day']=day
                    yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                    n=n+1
            if month == 2: #stupid february
                for day in range(1,29):
                    yrsum.loc[n,'Month']=month
                    yrsum.loc[n,'Day']=day
                    yrsum.loc[n,'Median']=data[(data[3]==day) & (data[2]==month)][4].median()
                    n=n+1
        ax1.semilogy(yrsum['Median']/QBar,'k-',alpha=0.4)
        
    plt.title('Typical Hydrographs for '+str(ngauges)+' Gauges in '+subsetname)
    ax1.set_ylabel('Median Daily Flow/Period Mean Flow')
    axes = plt.gca()
    axes.set_ylim(ylims)
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig

# Forcing Data

In [ ]:
def loadforcing(gauge,dataset='daymet'):
    """returns forcing data for given gauge from either 'daymet','maurer', or 'nldas'"""
    
    forcingfolder = home+'basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/basin_mean_forcing/'+dataset+'/'
    gaugeinfo = home + 'basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/basin_metadata/gauge_information.txt'

    gauges = pd.read_csv(gaugeinfo,sep='\t',header=0)  #had to adjust headers to line up right
    huc_id=gauges[gauges.GAGE_ID==gauge]['HUC_02'].values[0]

    GaugeInfo=gauges[gauges.GAGE_ID==gauge]

    if huc_id <10:
        nhuc = '0'+str(huc_id)
    else:
        nhuc = str(huc_id)

    forcings = forcingfolder+nhuc+'/'

    if dataset == 'daymet':
        label = 'cida'
    else:
        label=dataset

    if gauge < 10000000:
        data=forcings+'0'+str(gauge)+'_lump_'+label+'_forcing_leap.txt'
    else:
        data=forcings+str(gauge)+'_lump_'+label+'_forcing_leap.txt'

    data=pd.read_csv(data,sep='\s+',header=3)
    return data

def ForcingMinMedMax(gauge,columnname,dataset='daymet'):
    """return min/med/max plot for any of the forcing data over the years
    dataset 'daymet','maurer', or 'nldas'
    Dependent on loadforcing(gauge,dataset)"""

    data=loadforcing(gauge,dataset)
    data=data.replace(-999.0,np.nan)

    yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1

    fig, ax1 = plt.subplots()

    ax1.plot(yrsum['Max'],'b-',label='Max')
    ax1.plot(yrsum['Median'],'k-',label='Median')
    ax1.plot(yrsum['Min'],'r-',label='Min')
    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title(columnname+' Annual Variations at Gauge '+str(gauge))
    ax1.set_ylabel(columnname)
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig

def RasterPrecip(gauge,dataset='daymet'):
    """makes plot similar to raster hydrograph but of daily precipitation 
    dataset 'daymet','maurer', or 'nldas'   
    Dependent on loadforcing(gauge,dataset)"""

    columnname='prcp(mm/day)'
    
    data=loadforcing(gauge,dataset)
    
    data=data.replace(-999.0,np.nan)
    
    Years=data['Year'].unique()
    NYears=np.size(Years)
    RastData = np.empty([NYears,365])
    StartYear=Years[0]
    EndYear=Years[NYears-1]

    for nyear in range(0,NYears):
        yrdata=data[data['Year']==Years[nyear]][columnname].values
        if np.size(yrdata)==366: #leap years are a pain, let's just delete feb 29 from all years 
            yrdata=np.delete(yrdata,59)
        RastData[nyear,:]=yrdata

    RHscaled=RastData #np.log10(RastData+1) #making 0 flow days not break the log scale

    fig=plt.figure()
    shiftedcmap= plt.cm.cool
    shiftedcmap.set_under('white') 

    imgplot=plt.imshow(RHscaled,interpolation='nearest', aspect='auto',vmin=0.001) #,extent=[0,25,0,20])
    imgplot.set_cmap(shiftedcmap)
    plt.title('Gauge '+str(gauge)+', '+str(StartYear)+'-'+str(EndYear))
    plt.colorbar(label=columnname)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='major',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True) # labels along the bottom edge are off

    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    plt.xticks([15,45,74,105,135,166,196,227,258,288,319,349], xticklabels)
    labelnum=range(0,NYears,2)
    yticklabels=np.ones(np.size(labelnum),dtype=np.int16)
    for n in range(0,np.size(labelnum)):
        yticklabels[n]=StartYear+labelnum[n]
    plt.yticks(range(0,NYears,2), yticklabels)
    plt.show()
    return fig

def RasterTemps(gauge,maxmin,dataset='daymet'):
    """makes plot similar to raster hydrograph but of daily temperature max or min records 
    maxmin either = 'max' or 'min'
    dataset 'daymet','maurer', or 'nldas'
    Dependent on loadforcing(gauge,dataset)"""
    
    if maxmin=='max':
        columnname='tmax(C)' #'prcp(mm/day)'
    if maxmin=='min': 
        columnname='tmin(C)'
        
    data=loadforcing(gauge,dataset)

    data=data.replace(-999.0,np.nan)

    Years=data['Year'].unique()
    NYears=np.size(Years)
    RastData = np.empty([NYears,365])
    StartYear=Years[0]
    EndYear=Years[NYears-1]

    for nyear in range(0,NYears):
        yrdata=data[data['Year']==Years[nyear]][columnname].values
        if np.size(yrdata)==366: #leap years are a pain, let's just delete feb 29 from all years 
            yrdata=np.delete(yrdata,59)
        RastData[nyear,:]=yrdata

    RHscaled=RastData #np.log10(RastData+0.001) #making 0 flow days not break the log scale

    fig=plt.figure()
    colormap=plt.cm.bwr #coolwarm
    rangetemps=[np.max(RastData), np.min(RastData)]
    colorscaledata=rangetemps
    cmapmidpoint=1-max(colorscaledata)/(max(colorscaledata)+abs(min(colorscaledata)))
    shiftedcmap=shiftedColorMap(colormap, start=0, midpoint=cmapmidpoint, stop=1, name='shiftedcmap')


    imgplot=plt.imshow(RHscaled,interpolation='nearest', aspect='auto') #,extent=[0,25,0,20])
    imgplot.set_cmap(shiftedcmap)
    plt.title('Gauge '+str(gauge)+', '+str(StartYear)+'-'+str(EndYear))
    plt.colorbar(label=columnname)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='major',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True) # labels along the bottom edge are off

    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    plt.xticks([15,45,74,105,135,166,196,227,258,288,319,349], xticklabels)
    labelnum=range(0,NYears,2)
    yticklabels=np.ones(np.size(labelnum),dtype=np.int16)
    for n in range(0,np.size(labelnum)):
        yticklabels[n]=StartYear+labelnum[n]
    plt.yticks(range(0,NYears,2), yticklabels)
    plt.show()
    return fig

def RollingRasterPrecip(gauge,dataset='daymet',window=30, scalemax=[]):
    """Creates a raster plot of sum of n days previous precipitation for each day.
    Start dates without full window blacked out
    White if full window of no precipitation
    Dependent on loadforcing(gauge,dataset)"""
    
    data = loadforcing(gauge,dataset)
    
    columnname='RollingPrcp(mm/mo)'
    data[columnname]=data['prcp(mm/day)'].rolling(window).sum()

    Years=data['Year'].unique()
    NYears=np.size(Years)
    RastData = np.empty([NYears,365])
    StartYear=Years[0]
    EndYear=Years[NYears-1]

    for nyear in range(0,NYears):
        yrdata=data[data['Year']==Years[nyear]][columnname].values
        if np.size(yrdata)==366: #leap years are a pain, let's just delete feb 29 from all years 
            yrdata=np.delete(yrdata,59)
        RastData[nyear,:]=yrdata

    RHscaled=RastData #np.log10(RastData+1) #making 0 flow days not break the log scale

    fig=plt.figure()
    shiftedcmap= plt.cm.cool
    shiftedcmap.set_under('white') 
    shiftedcmap.set_bad('black') 


    if scalemax==[]: imgplot=plt.imshow(RHscaled,interpolation='nearest', aspect='auto',vmin=0.001) #,extent=[0,25,0,20])
    if scalemax!=[]: imgplot=plt.imshow(RHscaled,interpolation='nearest', aspect='auto',vmin=0.001,vmax=scalemax) #,extent=[0,25,0,20])

    imgplot.set_cmap(shiftedcmap)
    plt.title('Gauge '+str(gauge)+', '+str(StartYear)+'-'+str(EndYear))
    plt.colorbar(label=columnname)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='major',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True) # labels along the bottom edge are off

    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    plt.xticks([15,45,74,105,135,166,196,227,258,288,319,349], xticklabels)
    labelnum=range(0,NYears,2)
    yticklabels=np.ones(np.size(labelnum),dtype=np.int16)
    for n in range(0,np.size(labelnum)):
        yticklabels[n]=StartYear+labelnum[n]
    plt.yticks(range(0,NYears,2), yticklabels)
    plt.show()
    return fig

def RollingTypicalPrecip(gauge,dataset='daymet',window=30):
    """Gives min/med/max for precipitation over year given a window length
    Dependent on loadforcing(gauge,dataset)"""

    data = loadforcing(gauge,dataset)
    
    columnname='RollingPrcp(mm/mo)'
    data[columnname]=data['prcp(mm/day)'].rolling(window).sum()

    
    Years=data['Year'].unique()
    NYears=np.size(Years)
    RastData = np.empty([NYears,365])
    StartYear=Years[0]
    EndYear=Years[NYears-1]

    for nyear in range(0,NYears):
        yrdata=data[data['Year']==Years[nyear]][columnname].values
        if np.size(yrdata)==366: #leap years are a pain, let's just delete feb 29 from all years 
            yrdata=np.delete(yrdata,59)
        RastData[nyear,:]=yrdata

    yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1

    fig, ax1 = plt.subplots()
    ax1.plot(yrsum['Max'],'b-',label='Max')
    ax1.plot(yrsum['Median'],'k-',label='Median')
    ax1.plot(yrsum['Min'],'r-',label='Min')
    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title(columnname+' Annual Variations at '+str(gauge))
    ax1.set_ylabel(columnname)
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig

def RollingPrcp_select(gauge,selected_years,window=30,dataset='daymet'):
    """Plots rolling average precip for selected years over min/med/max for the period
    Dependent on loadforcing(gauge,dataset)"""
    
    data = loadforcing(gauge,dataset)

    columnname='RollingPrcp(mm/mo)'
    data[columnname]=data['prcp(mm/day)'].rolling(window).sum()


    yrsum= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsum.loc[n,'Month']=month
                yrsum.loc[n,'Day']=day
                yrsum.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsum.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsum.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1

    fig, ax1 = plt.subplots()
    ax1.plot(yrsum['Max'],'b-',label='Max',alpha=0.5)
    ax1.plot(yrsum['Median'],'k-',label='Median',alpha=0.5)
    ax1.plot(yrsum['Min'],'r-',label='Min',alpha=0.5)

    for year in selected_years:
        tempdata=data[data.Year == year]
        ax1.plot(range(0,365),tempdata[columnname],'-',label=str(year))

    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title(columnname+' Annual Variations at '+str(gauge))
    ax1.set_ylabel(columnname)
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig

def TypicalTemps(gauge,dataset='daymet'):
    """makes plot similar to Typical Year hydrograph but of daily temperature min/med/max records 
    dataset 'daymet','maurer', or 'nldas'
    Dependent on loadforcing(gauge,dataset)"""

    data=loadforcing(gauge,dataset)
    data=data.replace(-999.0,np.nan)

    columnname='tmax(C)'
    yrsumMAX= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsumMAX.loc[n,'Month']=month
                yrsumMAX.loc[n,'Day']=day
                yrsumMAX.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMAX.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMAX.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsumMAX.loc[n,'Month']=month
                yrsumMAX.loc[n,'Day']=day
                yrsumMAX.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMAX.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMAX.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsumMAX.loc[n,'Month']=month
                yrsumMAX.loc[n,'Day']=day
                yrsumMAX.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMAX.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMAX.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1

    columnname='tmin(C)'
    yrsumMIN= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsumMIN.loc[n,'Month']=month
                yrsumMIN.loc[n,'Day']=day
                yrsumMIN.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMIN.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMIN.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsumMIN.loc[n,'Month']=month
                yrsumMIN.loc[n,'Day']=day
                yrsumMIN.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMIN.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMIN.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsumMIN.loc[n,'Month']=month
                yrsumMIN.loc[n,'Day']=day
                yrsumMIN.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMIN.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMIN.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1

    fig, ax1 = plt.subplots()
    ax1.plot([0,365],[0,0],'k:')

    ax1.plot(yrsumMAX['Max'],'r-',label='MaxMax')
    ax1.plot(yrsumMAX['Median'],'k-',label='MedianMax',alpha=0.7)
    ax1.plot(yrsumMAX['Min'],'r-',label='MinMax',alpha=0.2)

    ax1.plot(yrsumMIN['Max'],'b-',label='MaxMin',alpha=0.2)
    ax1.plot(yrsumMIN['Median'],'k-',label='MedianMin',alpha=0.7)
    ax1.plot(yrsumMIN['Min'],'b-',label='MinMin')


    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title('Annual Temperature Variations at Gauge '+str(gauge))
    ax1.set_ylabel('Temperature (C)')
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig 

def TypicalTemp_selectyear(gauge,selectedyear,dataset='daymet'):
    """shows max and min temps for a single year over historic spread 
    (admittedly a bit slow)
    Dependent on loadforcing(gauge,dataset)"""

    data=loadforcing(gauge,dataset)
    data=data.replace(-999.0,np.nan)

    columnname='tmax(C)'
    yrsumMAX= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsumMAX.loc[n,'Month']=month
                yrsumMAX.loc[n,'Day']=day
                yrsumMAX.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMAX.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMAX.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsumMAX.loc[n,'Month']=month
                yrsumMAX.loc[n,'Day']=day
                yrsumMAX.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMAX.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMAX.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsumMAX.loc[n,'Month']=month
                yrsumMAX.loc[n,'Day']=day
                yrsumMAX.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMAX.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMAX.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1

    columnname='tmin(C)'
    yrsumMIN= pd.DataFrame(columns=['Month','Day','Median','Min','Max'])
    n=0
    for month in range(1,13,1):
        if month in [9,4,6,11]: #30 
            for day in range(1,31):
                yrsumMIN.loc[n,'Month']=month
                yrsumMIN.loc[n,'Day']=day
                yrsumMIN.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMIN.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMIN.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month in [1,3,5,7,8,10,12]: #31
            for day in range(1,32):
                yrsumMIN.loc[n,'Month']=month
                yrsumMIN.loc[n,'Day']=day
                yrsumMIN.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMIN.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMIN.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1
        if month == 2: #stupid february
            for day in range(1,29):
                yrsumMIN.loc[n,'Month']=month
                yrsumMIN.loc[n,'Day']=day
                yrsumMIN.loc[n,'Median']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].median()
                yrsumMIN.loc[n,'Min']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].min()
                yrsumMIN.loc[n,'Max']=data[(data['Day']==day) & (data['Mnth']==month)][columnname].max()
                n=n+1

    fig, ax1 = plt.subplots()
    ax1.plot([0,365],[0,0],'k:')
    

    ax1.plot(yrsumMAX['Max'],'r-',label='MaxMax',alpha=0.3)
    ax1.plot(yrsumMAX['Median'],'k-',label='MedianMax',alpha=0.3)

    ax1.plot(yrsumMIN['Median'],'k-',label='MedianMin',alpha=0.3)
    ax1.plot(yrsumMIN['Min'],'b-',label='MinMin',alpha=0.3)


    tempdata=data[data['Year'] == selectedyear]
    ax1.plot(range(0,365),tempdata['tmin(C)'],'b-',label=str(selectedyear))
    ax1.plot(range(0,365),tempdata['tmax(C)'],'r-',label=str(selectedyear))

    plt.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.title('Annual Temperature Variations at Gauge '+str(gauge))
    ax1.set_ylabel('Temperature (C)')
    xticklabels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    ax1.set_xticks([15,45,74,105,135,166,196,227,258,288,319,349],minor=False)
    ax1.set_xticks([0,31,59,90,120,151,181,212,243,273,304,334,365],minor=True)
    ax1.set_xticklabels(xticklabels)
    ax1.tick_params(axis='x',which='major',bottom=False,top=False,labelbottom=True)
    ax1.tick_params(axis='x',which='minor',bottom=True,top=False,labelbottom=False,width=-.5,length=18)
    plt.show()
    return fig

# Selected Gauges by Attributes

In [ ]:
def gaugefilter(subset,column,values):
    """Selects gauges that are within a range of values for a field in one of the attribute sets
    subset options: 
      'name','geol','clim','hydro','soil','topo','vege'
      'OtherAnalysis','fdc_info' 
    column is name of any numeric field in said subset
    values of form [min,max]
    dependent on returnattributeset(subset)"""
    
    #loading attributes/signatures 
    folder=home+'camels_attributes_v2.0/'; 

    df=returnattributeset(subset)
    
    filtered=df[(min(values)<df[column]) & (df[column]<max(values))]

    acceptable=set(filtered.iloc[:,0])
    return acceptable

def plotsquare(xrange,yrange,boxtype='k:',label=[]):
    """Adds square of x/y range onto other plot"""
    x=xrange[0],xrange[0],xrange[1],xrange[1],xrange[0]
    y=yrange[0],yrange[1],yrange[1],yrange[0],yrange[0]
    plt.plot(x,y,boxtype,label=label,alpha=0.8)

# Things other people made that are useful 

In [ ]:
import matplotlib
from mpl_toolkits.axes_grid1 import AxesGrid

def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    Thank you to Paul H on stackoverflow
    See https://stackoverflow.com/questions/7404116/defining-the-midpoint-of-a-colormap-in-matplotlib 
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap